This is a basic way of making an order using the API

In [ ]:
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
class TestApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)

    def nextValidId(self, orderId):
        # create contract
        contract = Contract()
        contract.symbol = "AAPL"
        contract.secType = "STK"
        contract.exchange = "SMART"
        contract.currency = "USD"
        contract.primaryExchange = "NASDAQ"

        # create order
        order = Order()
        order.action = "BUY"
        order.totalQuantity = 100
        order.orderType = "MKT"

        # place order
        self.placeOrder(orderId, contract, order)

def main():
    app = TestApp()
    app.connect("127.0.0.1", 7497, 0)
    app.run()

if __name__ == "__main__":
    main()


This is closer to what we will want to do:

In [5]:
import pandas as pd
from datetime import datetime
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order


class TestApp(EWrapper, EClient):
    """
    Trading application class that integrates EClient and EWrapper.
    """
    def __init__(self):
        EClient.__init__(self, self)
        self.order_status = {}
        self.total_orders = 0
        self.processed_orders = 0

    def nextValidId(self, orderId: int):
        """
        Callback function for a valid order ID.
        :param orderId: Next valid order ID.
        """
        print("Successfully connected to TWS. Next valid order ID:", orderId)
        self.orderId = orderId
        self.read_csv_and_trade("stocks.csv")

    def read_csv_and_trade(self, csv_file: str):
        """
        Read stock information from a CSV file and place trades.
        :param csv_file: Path to the CSV file.
        """
        df = pd.read_csv(csv_file)
        self.total_orders = len(df)
        for index, row in df.iterrows():
            self.trade_stock(row)


    def get_exchange_opening_time(self, exchange: str):
        """
        Get the opening time of an exchange.
        :param exchange: Name of the exchange.
        :return: datetime: Opening time of the exchange.
        """
        opening_times = {
            'NASDAQ': "09:30:00",  # 09:30:00 EST
            'NYSE': "09:30:00",    # 09:30:00 EST
            'LSE': "03:00:00",     # 08:00:00 GMT
            'TSE': "09:30:00",     # 09:30:00 EST (Toronto Stock Exchange)
            'CSE': "09:30:00",     # 09:30:00 EST (Canadian Securities Exchange)
            'ASX': "18:00:00",     # 10:00:00 AEDT (next day)
            'Euronext': "03:00:00", # 09:00:00 CET
            'Xetra': "03:00:00",   # 09:00:00 CET
            'SIX': "03:00:00"      # 09:00:00 CET (SIX Swiss Exchange)
        }

        today = datetime.now().strftime("%Y-%m-%d ")
        opening_time_str = opening_times.get(exchange, "")
        return datetime.strptime(today + opening_time_str, "%Y-%m-%d %H:%M:%S")

    def trade_stock(self, stock_info: pd.Series):
        """
        Place a trade for a stock.
        :param stock_info: Stock information.
        """
        # create contract
        contract = Contract()
        contract.symbol = stock_info['symbol']
        contract.secType = "STK"
        contract.exchange = stock_info['exchange']
        contract.currency = stock_info['currency']
        contract.primaryExchange = stock_info['primaryExchange']

        # create order
        order = Order()
        # These are no longer supported and are default true
        order.eTradeOnly = False
        order.firmQuoteOnly = False

        order.action = stock_info['action']
        order.totalQuantity = int(stock_info['quantity'])
        order.orderType = "MKT"

        # place order
        self.placeOrder(self.orderId, contract, order)
        self.orderId += 1

    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice,
                    permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        """
        Callback function for receiving order status updates.
        """
        if orderId not in self.order_status or self.order_status[orderId] != status:
            print(f"Order ID: {orderId}, Status: {status}")
            self.order_status[orderId] = status  # Update the status when it changes
            self.processed_orders += 1

            if self.processed_orders >= self.total_orders:
                self.disconnect()

    def openOrder(self, orderId, contract, order, orderState):
        """
        Callback function for open orders.
        """
        if orderId not in self.order_status or self.order_status[orderId] != orderState.status:
            print(f"Open Order ID: {orderId}, {contract.symbol}, {contract.secType} at {contract.exchange}, Status: {orderState.status}")
            self.order_status[orderId] = orderState.status  # Update the status

def main():
    app = TestApp()
    app.connect("127.0.0.1", 7497, 0)
    app.run()

if __name__ == "__main__":
    main()


ERROR -1 502 Couldn't connect to TWS. Confirm that "Enable ActiveX and Socket EClients" 
is enabled and connection port is the same as "Socket Port" on the 
TWS "Edit->Global Configuration...->API->Settings" menu. Live Trading ports: 
TWS: 7496; IB Gateway: 4001. Simulated Trading ports for new installations 
of version 954.1 or newer:  TWS: 7497; IB Gateway: 4002
